In [2]:
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection 
import matplotlib.colors as mcolors
from matplotlib import cm
from scipy import stats

import numpy as np
import sys
import trimesh
###
# Asteroid Name
asteroid = '1950DA_Prograde'

########################################################
#################################### Personal Packages #    
sys.dont_write_bytecode = True
import constants as C
const    = C.constants()
target = C.DA1950()
Spin = target.spin
# 
omega = (2*np.pi)/(Spin * 3600)
###########################################################
################################################ Load files
# object file
obj_Path =  asteroid + '.obj' 
# MASCONs (tetrahedron center of masses)
CM_Path =  asteroid + '_CM.in' 
CM = np.loadtxt(CM_Path, delimiter=' ',dtype=float)
Terta_Count = len(CM)
#######################################
# Polyhedron 
mesh = trimesh.load_mesh(obj_Path)
# Define the scale factor
mesh.apply_scale(target.gamma)
# Calculate the volume of the polyhedron
polyhedron_volume = mesh.volume
R_eff = (3 * polyhedron_volume / (4 * np.pi)) ** (1/3)
print(f"Volume: {polyhedron_volume} km^3")
print(f"Effective Radius: {R_eff} km")
########################################
# Gravitational Parameter of each MASCON
mu_Path =   asteroid + '_mu.in'
mu_I = np.loadtxt(mu_Path, delimiter=' ')
mu = np.sum(mu_I)
G = const.G
Mass = mu/G
den = Mass/polyhedron_volume
print(f"Total Gravitational Parameter: {mu}")
print(f"Total Mass: {Mass:.5e} kg")
print(f"Density: {den} kg/km^3")
#######################################################
#######################################################
all_z = []
scatter_plots = []
mesh = trimesh.load_mesh('Apophis.obj')
gamma = 0.285 
v = mesh.vertices*gamma
f = mesh.faces - 1
print(f"Vertices: {v.shape}")
print(f"Faces: {f.shape}") 

Volume: 0.7978950017513031 km^3
Effective Radius: 0.5753768422177097 km
Total Gravitational Parameter: 1.2780937464839698e-07
Total Mass: 1.91495e+12 kg
Density: 2400007191859.606 kg/km^3
Vertices: (1014, 3)
Faces: (2024, 3)


In [23]:
file1   = 'Databank/1950DA/Landing/Landing_velocities.dat' 
########
landings = np.loadtxt(file1, delimiter=' ',dtype=float)


a0 = landings[:,0]
H  = landings[:,1]
Dv = landings[:,2]
x  = landings[:,3]
y  = landings[:,4]
z  = landings[:,5]
vx = landings[:,6]
vy = landings[:,7]  
vz = landings[:,8]
########################################
min_Dv = np.min(Dv)
max_Dv = np.max(Dv)
#
min_id = np.argmin(Dv)
print(f"Minimum Delta-V: {min_Dv} km/s")
print(f"At a0 = {a0[min_id]} km")
print(f"At H = {H[min_id]}")
print(f"At position: ({x[min_id]}, {y[min_id]}, {z[min_id]}) km")
print(f"At velocity: ({vx[min_id]}, {vy[min_id]}, {vz[min_id]}) km/s")
########################################
Filter_Ld = []
for i in range(0, len(a0)):
    if a0[i] > 1.2 and a0[i] < 3.1:
        Filter_Ld.append(landings[i,:])
        #print(Filter_Ld)
########################################
filt_Dv = np.array(Filter_Ld)[:,2]
min_filt_Dv = np.min(filt_Dv)
min_filt_id = np.argmin(filt_Dv)
print(f"Minimum Filtered Delta-V: {min_filt_Dv} km/s")
print(f"At a0 = {Filter_Ld[min_filt_id][0]} km")


Minimum Delta-V: 0.0006994519744693233 km/s
At a0 = 0.69 km
At H = 2.8000000000000003e-08
At position: (-0.049400619404999985, 0.08271789032429672, -0.5866377027161449) km
At velocity: (0.0004700954232193101, 0.0003682240191192799, 0.0003642175577878375) km/s
Minimum Filtered Delta-V: 0.0007135026484743678 km/s
At a0 = 1.52 km


In [25]:
file1   = 'Databank/1950DA/Landing2/Landing_velocities.dat' 
########
landings = np.loadtxt(file1, delimiter=' ',dtype=float)


a0 = landings[:,0]
H  = landings[:,1]
Dv = landings[:,2]
x  = landings[:,3]
y  = landings[:,4]
z  = landings[:,5]
vx = landings[:,6]
vy = landings[:,7]  
vz = landings[:,8]
########################################
min_Dv = np.min(Dv)
max_Dv = np.max(Dv)
#
min_id = np.argmin(Dv)
print(f"Minimum Delta-V: {min_Dv} km/s")
print(f"At a0 = {a0[min_id]} km")
print(f"At H = {H[min_id]}")
print(f"At position: ({x[min_id]}, {y[min_id]}, {z[min_id]}) km")
print(f"At velocity: ({vx[min_id]}, {vy[min_id]}, {vz[min_id]}) km/s")
########################################
Filter_Ld = []
for i in range(0, len(a0)):
    if a0[i] ==3.0 :
        Filter_Ld.append(landings[i,:])
        #print(Filter_Ld)
########################################
filt_Dv = np.array(Filter_Ld)[:,2]
min_filt_Dv = np.min(filt_Dv)
min_filt_id = np.argmin(filt_Dv)
print(f"Minimum Filtered Delta-V: {min_filt_Dv} km/s")
print(f"At a0 = {Filter_Ld[min_filt_id][0]} km")
print(f"At H = {Filter_Ld[min_filt_id][1]}")

Minimum Delta-V: 0.0007262876151993389 km/s
At a0 = 2.5 km
At H = 3.0000000000000004e-08
At position: (-0.2115746728362357, -0.014567118897384587, -0.5370145485060904) km
At velocity: (0.00035926313015246267, 0.0004849553497750865, 0.00040403219182327674) km/s
Minimum Filtered Delta-V: 0.0007585244218228494 km/s
At a0 = 3.0 km
At H = 2e-08


In [26]:
def v_calc(Ham,omega,mu_I,CM,yp):
    U = np.zeros(1, dtype="float64")
    for it in range(len(CM)):
        xi = 0  - CM[it,0]
        yi = yp - CM[it,1]
        zi = 0  - CM[it,2]
        rho = np.sqrt(xi**2 + yi**2 + zi**2) 
        # r = np.sqrt(y**2)
        U += mu_I[it]/rho
    #########################
    psu = U[0]
    centri = (omega**2)*(yp**2)
    # print(f"Omega: {omega} rad/s")
    # print(f"Ham:      {Ham} (km^2/s^2) ")
    # print(f"Psuedo:   {psu} (km^2/s^2) ")
    # print(f"Centrifu: {centri} (km^2/s^2) ")
    arg = 2*Ham + centri + 2*psu
    if arg < 0.0:
        V = np.nan
    else:
        V = np.sqrt(arg)
    return V

In [29]:
v_negri = v_calc(2.0e-6,omega,mu_I,CM,3.0)
print(f"Negri Velocity at 3.0 km: {v_negri} km/s")
########################################
v_lower = v_calc(2.0e-8,omega,mu_I,CM,3.0)
print(f"Lower Bound Velocity at 3.0 km: {v_lower} km/s")
########################################
delV = abs(v_negri - v_lower)
print(f"Delta-V between Negri and Lower Bound at 3.0 km: {delV} km/s")
#########################################
total_land_Dv = delV + min_filt_Dv
print(f"Total Delta-V at 3.0 km: {total_land_Dv} km/s")

Negri Velocity at 3.0 km: 0.0031900054788011993 km/s
Lower Bound Velocity at 3.0 km: 0.002493217791285324 km/s
Delta-V between Negri and Lower Bound at 3.0 km: 0.0006967876875158753 km/s
Total Delta-V at 3.0 km: 0.0014553121093387248 km/s


In [1]:
print(f"Asteroid Navigator \u2609-\u263F-\u2640-\u2295\u263D-\u2642-\u2643-\u2644-\u26E2-\u2646-\u2647")

Asteroid Navigator ☉-☿-♀-⊕☽-♂-♃-♄-⛢-♆-♇
